In [4]:
import matplotlib.pyplot as plt

from collections import OrderedDict
from sklearn.datasets import make_classification 
from sklearn.ensemble import RandomForestClassifier
RANDOM_STATE=0

X, y = make_classification(n_samples=1500, n_features=25,
                           n_clusters_per_class=1, n_informative=15,
                           random_state=RANDOM_STATE)
clf=RandomForestClassifier(n_estimators=100,
                       warm_start=True, oob_score=True,
                       max_features="sqrt",
                       random_state=RANDOM_STATE)
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=True, random_state=0, verbose=0, warm_start=True)

In [5]:
clf.estimators_[0].tree_.children_right

array([118,  53,  38,   9,   8,   7,  -1,  -1,  -1,  29,  20,  19,  18,
        17,  16,  -1,  -1,  -1,  -1,  -1,  28,  27,  24,  -1,  26,  -1,
        -1,  -1,  -1,  37,  34,  33,  -1,  -1,  36,  -1,  -1,  -1,  52,
        41,  -1,  43,  -1,  51,  48,  47,  -1,  -1,  50,  -1,  -1,  -1,
        -1,  95,  60,  57,  -1,  59,  -1,  -1,  84,  75,  66,  65,  -1,
        -1,  74,  69,  -1,  73,  72,  -1,  -1,  -1,  -1,  79,  78,  -1,
        -1,  81,  -1,  83,  -1,  -1,  86,  -1,  94,  93,  90,  -1,  92,
        -1,  -1,  -1,  -1, 107, 106, 105, 104, 103, 102,  -1,  -1,  -1,
        -1,  -1,  -1, 109,  -1, 111,  -1, 113,  -1, 115,  -1, 117,  -1,
        -1, 196, 181, 166, 141, 134, 131, 130, 129, 128,  -1,  -1,  -1,
        -1, 133,  -1,  -1, 136,  -1, 138,  -1, 140,  -1,  -1, 159, 158,
       147, 146,  -1,  -1, 151, 150,  -1,  -1, 157, 154,  -1, 156,  -1,
        -1,  -1,  -1, 165, 164, 163,  -1,  -1,  -1,  -1, 168,  -1, 172,
       171,  -1,  -1, 180, 177, 176,  -1,  -1, 179,  -1,  -1,  -

In [6]:
import sklearn

help(sklearn.tree._tree.Tree)


Help on class Tree in module sklearn.tree._tree:

class Tree(builtins.object)
 |  Array-based representation of a binary decision tree.
 |  
 |  The binary tree is represented as a number of parallel arrays. The i-th
 |  element of each array holds information about the node `i`. Node 0 is the
 |  tree's root. You can find a detailed description of all arrays in
 |  `_tree.pxd`. NOTE: Some of the arrays only apply to either leaves or split
 |  nodes, resp. In this case the values of nodes of the other type are
 |  arbitrary!
 |  
 |  Attributes
 |  ----------
 |  node_count : int
 |      The number of nodes (internal nodes + leaves) in the tree.
 |  
 |  capacity : int
 |      The current capacity (i.e., size) of the arrays, which is at least as
 |      great as `node_count`.
 |  
 |  max_depth : int
 |      The maximal depth of the tree.
 |  
 |  children_left : array of int, shape [node_count]
 |      children_left[i] holds the node id of the left child of node i.
 |      For leaves,

In [47]:
def tree_extraction_cost(estimator,costs):
    features_used= set(estimator.tree_.feature)
    cost=0
    for f in features_used:
        cost+=costs[f]
    return cost

def rf_extraction_cost(rf,costs):
    features_used= set()
    for estimator in rf.estimators_:
        features_used=features_used.union(set(estimator.tree_.feature))
    features_used.remove(-2)
    cost=0
    for f in features_used:
        cost+=costs[f]
    return cost

def tree_evaluation_cost(estimator,node=0,depth=1):
    if estimator.tree_.children_left[node]==-1 :
        return estimator.tree_.weighted_n_node_samples[node]*depth
    return tree_evaluation_cost(estimator,estimator.tree_.children_left[node],depth+1)+\
           tree_evaluation_cost(estimator,estimator.tree_.children_right[node],depth+1)
    
def forest_evaluation_cost(rf):
    cost=0
    for estimator in rf.estimators_:
        cost+=tree_evaluation_cost(estimator)
tree_evaluation_cost(clf.estimators_[0])

def tree_budget_exact(estimator):
    tree_evaluation_cost(estimator)+tree_extraction_cost(estimator)
    
def forest_budget_exact(rf):
    forest_evaluation_cost(rf)+forest_extraction_cost(rf)
    
def tree_budget_rough(estimator,tree_cost=1):
    tree_cost+tree_extraction_cost(estimator)
    
def forest_budget_rough(rf,tree_cost=1):
    tree_cost*rf.n_estimators+forest_extraction_cost(rf)    